In [45]:
import pandas as pd
import numpy as np

In [63]:
#make a dataframe from AMS_descritt .csv file

df = pd.read_csv('AMS_descritt_noSM_meltD_adim.csv', sep=',', encoding='utf-8')
df.drop(df.columns[[0]], axis=1, inplace=True)
df

,ID,Nome,HMerit_mean(m),HMerit_stdev(m),Slope_mean(deg),Slope_stdev(deg),Aspect_mean(deg),Aspect_stdev(deg),MinDistAdriaticC_(km),OrogrAdri_mean (m),...,X,Y,duration[h],AMS,mean_IdD,loc,scale,99perc,95perc,80perc
0,5001,'Abetone',1319.144154,85.599127,13.342638,4.979222,252.769653,96.036924,134.037400,377.648461,...,633069.81,4890553.68,1,1.221239,28.250000,0.890564,0.184415,1.738901,1.438313,1.167176
1,5001,'Abetone',1319.144154,85.599127,13.342638,4.979222,252.769653,96.036924,134.037400,377.648461,...,633069.81,4890553.68,1,0.778761,28.250000,0.890564,0.184415,1.738901,1.438313,1.167176
2,5002,'Albareto',531.112211,40.321631,7.545648,4.469855,219.061317,121.198571,203.680776,300.679959,...,555576.92,4921743.21,1,1.219018,33.469565,0.823918,0.287186,2.145015,1.676915,1.254679
3,5002,'Albareto',531.112211,40.321631,7.545648,4.469855,219.061317,121.198571,203.680776,300.679959,...,555576.92,4921743.21,1,0.979995,33.469565,0.823918,0.287186,2.145015,1.676915,1.254679
4,5002,'Albareto',531.112211,40.321631,7.545648,4.469855,219.061317,121.198571,203.680776,300.679959,...,555576.92,4921743.21,1,1.153287,33.469565,0.823918,0.287186,2.145015,1.676915,1.254679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274542,21338,'Vicenza_-_Sant_Agostino',33.290198,3.489680,0.844974,1.510106,172.196092,94.760327,66.170105,15.368078,...,696822.00,5044294.00,24,0.918877,82.927273,0.903636,0.193259,1.792657,1.477653,1.193513
274543,21338,'Vicenza_-_Sant_Agostino',33.290198,3.489680,0.844974,1.510106,172.196092,94.760327,66.170105,15.368078,...,696822.00,5044294.00,24,0.950230,82.927273,0.903636,0.193259,1.792657,1.477653,1.193513
274544,21338,'Vicenza_-_Sant_Agostino',33.290198,3.489680,0.844974,1.510106,172.196092,94.760327,66.170105,15.368078,...,696822.00,5044294.00,24,0.602938,82.927273,0.903636,0.193259,1.792657,1.477653,1.193513
274545,21338,'Vicenza_-_Sant_Agostino',33.290198,3.489680,0.844974,1.510106,172.196092,94.760327,66.170105,15.368078,...,696822.00,5044294.00,24,0.877878,82.927273,0.903636,0.193259,1.792657,1.477653,1.193513


In [60]:
#quanti sono gli ID unici?

df['ID'].nunique()

2352

In [64]:
grouped_AMS = df.groupby(['ID', 'duration[h]'])['AMS'].apply(list).reset_index()
grouped_IdD = df.groupby(['ID', 'duration[h]'])['mean_IdD'].unique().apply(lambda x:  x[0] if len(x) > 0 else None).reset_index()

pivoted_AMS = grouped_AMS.pivot(index='ID', columns='duration[h]', values='AMS')
pivoted_IdD = grouped_IdD.pivot(index='ID', columns='duration[h]', values='mean_IdD')

pivoted_AMS.columns = [f'AMS_{col}' for col in pivoted_AMS.columns]
pivoted_IdD.columns = [f'IdD_{col}' for col in pivoted_IdD.columns]
 
pivoted_AMS.reset_index(inplace=True)
pivoted_IdD.reset_index(inplace=True)

df = pd.merge(df, pivoted_AMS, on='ID')
df = pd.merge(df, pivoted_IdD, on='ID')

df

,ID,Nome,HMerit_mean(m),HMerit_stdev(m),Slope_mean(deg),Slope_stdev(deg),Aspect_mean(deg),Aspect_stdev(deg),MinDistAdriaticC_(km),OrogrAdri_mean (m),...,AMS_1,AMS_3,AMS_6,AMS_12,AMS_24,IdD_1,IdD_3,IdD_6,IdD_12,IdD_24
0,5001,'Abetone',1319.144154,85.599127,13.342638,4.979222,252.769653,96.036924,134.037400,377.648461,...,"[1.2212389380530972, 0.7787610619469026]","[1.1067961165048543, 0.8932038834951457, 1.184...","[1.136231884057971, 0.9391304347826088, 1.1130...","[1.2430555555555556, 0.9861111111111112, 1.041...","[1.5184490306441525, 0.9455909943714822, 0.855...",28.25,51.500,86.25,144.000,199.8750
1,5001,'Abetone',1319.144154,85.599127,13.342638,4.979222,252.769653,96.036924,134.037400,377.648461,...,"[1.2212389380530972, 0.7787610619469026]","[1.1067961165048543, 0.8932038834951457, 1.184...","[1.136231884057971, 0.9391304347826088, 1.1130...","[1.2430555555555556, 0.9861111111111112, 1.041...","[1.5184490306441525, 0.9455909943714822, 0.855...",28.25,51.500,86.25,144.000,199.8750
2,5001,'Abetone',1319.144154,85.599127,13.342638,4.979222,252.769653,96.036924,134.037400,377.648461,...,"[1.2212389380530972, 0.7787610619469026]","[1.1067961165048543, 0.8932038834951457, 1.184...","[1.136231884057971, 0.9391304347826088, 1.1130...","[1.2430555555555556, 0.9861111111111112, 1.041...","[1.5184490306441525, 0.9455909943714822, 0.855...",28.25,51.500,86.25,144.000,199.8750
3,5001,'Abetone',1319.144154,85.599127,13.342638,4.979222,252.769653,96.036924,134.037400,377.648461,...,"[1.2212389380530972, 0.7787610619469026]","[1.1067961165048543, 0.8932038834951457, 1.184...","[1.136231884057971, 0.9391304347826088, 1.1130...","[1.2430555555555556, 0.9861111111111112, 1.041...","[1.5184490306441525, 0.9455909943714822, 0.855...",28.25,51.500,86.25,144.000,199.8750
4,5001,'Abetone',1319.144154,85.599127,13.342638,4.979222,252.769653,96.036924,134.037400,377.648461,...,"[1.2212389380530972, 0.7787610619469026]","[1.1067961165048543, 0.8932038834951457, 1.184...","[1.136231884057971, 0.9391304347826088, 1.1130...","[1.2430555555555556, 0.9861111111111112, 1.041...","[1.5184490306441525, 0.9455909943714822, 0.855...",28.25,51.500,86.25,144.000,199.8750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274542,9392,'Salo',85.597313,31.713678,3.567789,5.443041,139.705672,128.961728,142.805883,125.923719,...,NaN,"[0.7737089201877935, 0.7737089201877935, 0.758...","[0.7651715039577837, 0.633245382585752, 0.8865...","[0.7401662049861496, 0.8421052631578947, 1.178...","[0.791792395896198, 0.8529470931402132, 1.2005...",NaN,26.625,37.90,45.125,62.1375
274543,9392,'Salo',85.597313,31.713678,3.567789,5.443041,139.705672,128.961728,142.805883,125.923719,...,NaN,"[0.7737089201877935, 0.7737089201877935, 0.758...","[0.7651715039577837, 0.633245382585752, 0.8865...","[0.7401662049861496, 0.8421052631578947, 1.178...","[0.791792395896198, 0.8529470931402132, 1.2005...",NaN,26.625,37.90,45.125,62.1375
274544,18116,'Bolzano_Genio_Civile',474.330294,143.001570,21.666254,11.455896,184.335876,71.201920,145.508069,661.757866,...,NaN,NaN,[1.0],[1.0],[1.0],NaN,NaN,22.00,26.800,27.8000
274545,18116,'Bolzano_Genio_Civile',474.330294,143.001570,21.666254,11.455896,184.335876,71.201920,145.508069,661.757866,...,NaN,NaN,[1.0],[1.0],[1.0],NaN,NaN,22.00,26.800,27.8000


In [65]:
#take only the rows with unique ID

df_unique = df.drop_duplicates(subset=['ID'], keep='first')
df_unique.drop(['duration[h]','AMS', 'mean_IdD'], axis=1, inplace=True)
df_unique.reset_index(drop=True, inplace=True)
df_unique

/var/folders/73/q15nk7zj12b12wgzsnb2wkwm0000gn/T/ipykernel_1415/4171108170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(['duration[h]','AMS', 'mean_IdD'], axis=1, inplace=True)


,ID,Nome,HMerit_mean(m),HMerit_stdev(m),Slope_mean(deg),Slope_stdev(deg),Aspect_mean(deg),Aspect_stdev(deg),MinDistAdriaticC_(km),OrogrAdri_mean (m),...,AMS_1,AMS_3,AMS_6,AMS_12,AMS_24,IdD_1,IdD_3,IdD_6,IdD_12,IdD_24
0,5001,'Abetone',1319.144154,85.599127,13.342638,4.979222,252.769653,96.036924,134.037400,377.648461,...,"[1.2212389380530972, 0.7787610619469026]","[1.1067961165048543, 0.8932038834951457, 1.184...","[1.136231884057971, 0.9391304347826088, 1.1130...","[1.2430555555555556, 0.9861111111111112, 1.041...","[1.5184490306441525, 0.9455909943714822, 0.855...",28.250000,51.500000,86.250000,144.000000,199.875000
1,5002,'Albareto',531.112211,40.321631,7.545648,4.469855,219.061317,121.198571,203.680776,300.679959,...,"[1.2190179267342167, 0.9799948038451548, 1.153...","[0.8760114425827543, 1.0752758479771147, 1.387...","[0.6966149147365742, 0.9015016543649784, 1.211...","[0.8087813881198699, 1.0254990528923211, 0.929...","[0.7013766607971826, 0.9903953897870976, 0.920...",33.469565,53.195652,68.330435,89.517391,108.643478
2,5003,'Alberino',9.449215,1.201387,0.300186,0.212229,170.173093,96.225534,47.001907,0.532786,...,"[1.010657846380007, 0.7883131201764058, 1.2936...","[1.0150596252129471, 0.9311073253833048, 2.136...","[1.051613628696728, 1.196446643427415, 1.89542...","[0.8763774423557736, 1.1978886933975368, 1.617...","[1.0819864825154275, 0.9667940052894504, 1.324...",19.789091,26.205357,31.760714,38.567857,48.614286
3,5004,'Alfonsine',4.965529,0.633608,0.166332,0.099378,164.227651,85.722168,18.849369,2.217621,...,"[1.3696369636963697, 1.3283828382838283, 0.924...","[1.361474435196195, 1.521997621878716, 0.93341...","[1.8906211333828264, 1.291759465478842, 0.7869...","[2.233708179087797, 1.110567856394496, 0.69567...","[2.1208944793850453, 0.9993011879804332, 0.719...",24.240000,33.640000,40.410000,47.723333,57.240000
4,5005,'Anzola_Dellemilia',32.734555,1.486335,0.250642,0.153502,137.487546,105.544861,84.801872,8.606806,...,"[1.600310941245061, 2.139275766016713, 0.82088...","[1.2349222945677487, 1.837175039209163, 0.6022...","[1.514415897691159, 1.4801087414327834, 0.6518...","[1.2947304663840096, 1.1706844336765596, 1.015...","[1.0473040836824183, 1.1445089537247986, 1.194...",24.120313,32.876562,40.807812,51.593750,63.782812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,21337,'Verona_-_Parco_Adige_Nord',72.569159,8.564774,1.836605,2.212769,174.910397,79.410425,107.642870,45.148810,...,"[0.9292502639915522, 1.620380147835269, 0.9699...","[0.7630331753554503, 1.4928909952606635, 1.037...","[0.6950549450549451, 1.3901098901098905, 1.079...","[0.6023809523809525, 1.204761904761905, 1.1561...","[0.6224662162162163, 1.170608108108108, 1.1365...",34.436364,42.200000,46.327273,53.454545,64.581818
2348,21338,'Vicenza_-_Sant_Agostino',33.290198,3.489680,0.844974,1.510106,172.196092,94.760327,66.170105,15.368078,...,"[1.1151439299123902, 1.1908635794743427, 0.936...","[0.8774627582892841, 1.3267659778952428, 0.792...","[0.7569546120058566, 1.1072474377745245, 0.946...","[1.0882518268690278, 1.128442945474986, 1.0078...","[1.0828765621574217, 1.126288094716071, 0.9743...",29.054545,37.836364,49.672727,64.690909,82.927273
2349,5209,'Nirone_Diga',690.596481,85.467817,16.493247,7.704602,216.538439,99.727580,163.812109,207.441075,...,NaN,"[1.147540983606557, 0.8524590163934426]","[1.2577319587628866, 0.7422680412371134]",[1.0],"[1.1016042780748665, 0.8983957219251337]",NaN,30.500000,48.500000,83.000000,93.500000
2350,9392,'Salo',85.597313,31.713678,3.567789,5.443041,139.705672,128.961728,142.805883,125.923719,...,NaN,"[0.7737089201877935, 0.7737089201877935, 0.758...","[0.7651715039577837, 0.633245382585752, 0.8865...","[0.7401662049861496, 0.8421052631578947, 1.178...","[0.791792395896198, 0.8529470931402132, 1.2005...",NaN,26.625000,37.900000,45.125000,62.137500


In [66]:
df2 = pd.read_csv('gumMap_statbench_Gumfit_NEW.csv', sep=',', encoding='utf-8')
df2

,Unnamed: 0,ID,Nome,md_1h,md_3h,md_6h,md_12h,md_24h,q1h_0.80,q1h_0.95,...,q_3h_0.99,q_6h_0.80,q_6h_0.95,q_6h_0.99,q_12h_0.80,q_12h_0.95,q_12h_0.99,q_24h_0.80,q_24h_0.95,q_24h_0.99
0,0,5005,'Anzola_Dellemilia',1.0,1.0,1.0,1.0,1.0,1.297268,1.770921,...,2.204652,1.262136,1.679814,2.142859,1.256130,1.664236,2.116671,1.257223,1.667072,2.121439
1,1,5007,'Argelato',1.0,1.0,1.0,1.0,1.0,1.307714,1.798013,...,2.270210,1.272879,1.707674,2.189696,1.264559,1.686097,2.153422,1.257970,1.669008,2.124694
2,2,5015,'Bardi_Centrale',1.0,1.0,1.0,1.0,1.0,1.271656,1.704500,...,2.174044,1.257236,1.667106,2.121495,1.251259,1.651604,2.095435,1.253316,1.656940,2.104405
3,3,5141,'Invaso',1.0,1.0,1.0,1.0,1.0,1.271679,1.704561,...,2.174045,1.257236,1.667106,2.121496,1.251259,1.651605,2.095436,1.253324,1.656960,2.104438
4,4,5143,'Isola_Di_Palanzano_Centrale',1.0,1.0,1.0,1.0,1.0,1.265930,1.689653,...,2.173967,1.257225,1.667077,2.121446,1.251233,1.651538,2.095324,1.250087,1.648565,2.090326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,21020,'Buttapietra',1.0,1.0,1.0,1.0,1.0,1.293367,1.760805,...,2.192209,1.260123,1.674593,2.134083,1.254338,1.659590,2.108860,1.256884,1.666191,2.119958
110,110,21060,'Grantorto',1.0,1.0,1.0,1.0,1.0,1.275492,1.714451,...,2.174355,1.257283,1.667228,2.121701,1.251340,1.651814,2.095787,1.254370,1.659671,2.108997
111,111,21068,'Lusia',1.0,1.0,1.0,1.0,1.0,1.300470,1.779225,...,2.219019,1.264475,1.685879,2.153055,1.258088,1.669313,2.125207,1.257474,1.667723,2.122532
112,112,21267,'Rovigo_Simn',1.0,1.0,1.0,1.0,1.0,1.303783,1.787819,...,2.238802,1.267712,1.694274,2.167168,1.260662,1.675989,2.136429,1.257712,1.668339,2.123569


In [67]:
#take the first 100 ID from df2 and use them to create the test set from df_unique

test_ID = df2['ID'].to_list()[:100]
test_df = df_unique[df_unique['ID'].isin(test_ID)]
test_df.reset_index(drop=True, inplace=True)
test_df

,ID,Nome,HMerit_mean(m),HMerit_stdev(m),Slope_mean(deg),Slope_stdev(deg),Aspect_mean(deg),Aspect_stdev(deg),MinDistAdriaticC_(km),OrogrAdri_mean (m),...,AMS_1,AMS_3,AMS_6,AMS_12,AMS_24,IdD_1,IdD_3,IdD_6,IdD_12,IdD_24
0,5005,'Anzola_Dellemilia',32.734555,1.486335,0.250642,0.153502,137.487546,105.544861,84.801872,8.606806,...,"[1.600310941245061, 2.139275766016713, 0.82088...","[1.2349222945677487, 1.837175039209163, 0.6022...","[1.514415897691159, 1.4801087414327834, 0.6518...","[1.2947304663840096, 1.1706844336765596, 1.015...","[1.0473040836824183, 1.1445089537247986, 1.194...",24.120313,32.876562,40.807812,51.593750,63.782812
1,5007,'Argelato',20.012779,0.978248,0.242489,0.159772,183.111363,113.340764,71.044891,4.043628,...,"[1.3579433091628212, 1.230498791474401, 1.5820...","[1.065700982928091, 0.9656837385756166, 2.0348...","[0.9520984724899898, 0.8779475011122647, 2.206...","[0.8274231678486998, 0.8983451536643027, 1.768...","[0.643510054844607, 1.1809872029250454, 1.3857...",22.755000,28.995000,33.715000,42.300000,54.700000
2,5015,'Bardi_Centrale',477.632546,49.854183,11.375176,6.982993,171.387509,111.067833,200.889706,152.467934,...,"[1.6646836127636562, 1.2947539210383991, 1.191...","[1.3969016092126487, 1.2828688247871265, 0.769...","[1.0769230769230769, 1.2384615384615385, 0.538...","[0.8822974926161001, 1.4133098724313455, 0.767...","[0.8455575041384622, 1.4513300444167636, 1.110...",24.328947,35.077632,46.428571,61.203846,79.237662
3,5141,'Invaso',548.981944,65.788483,12.771520,5.341303,160.901902,97.866923,88.955157,191.483851,...,"[0.8746962860118014, 0.641443943075321, 0.5581...","[1.042436345481777, 0.5871193210184723, 0.5032...","[1.2374044305038228, 0.4752009409919623, 0.526...","[1.2775232980571791, 0.5383035855315116, 0.689...","[1.7874875868917577, 0.7239324726911618, 0.697...",24.008333,33.383333,42.508333,52.758333,67.133333
4,5143,'Isola_Di_Palanzano_Centrale',723.125769,92.912061,17.467407,8.735623,191.868201,105.323390,167.619090,176.588696,...,"[0.9172717046732796, 1.1642294713160857, 0.493...","[0.7972940323749698, 0.8697753080454216, 0.555...","[0.7863906275076232, 0.898732145722998, 0.6553...","[0.6946413382478027, 0.793875815140346, 0.5788...","[0.6852126685402128, 1.3704253370804256, 0.906...",28.344928,41.390000,53.408571,60.462857,90.482857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,16580,'Pietramala',878.285624,68.940387,10.479020,6.632870,143.437356,47.537888,82.012179,224.094045,...,"[1.3025499655410062, 0.8270158511371468, 1.157...","[1.618454778469425, 0.8604906627608934, 1.1387...","[1.5675675675675675, 1.003861003861004, 1.1943...","[1.5370196813495782, 0.9489222118088098, 1.220...","[1.4542728635682158, 0.9873634611265796, 1.126...",29.020000,54.620000,77.700000,85.360000,93.380000
96,17011,'Rovereto',236.555646,57.446341,6.717584,6.781555,257.745566,57.691170,116.652683,290.435894,...,"[1.125125572050452, 1.4224801875209288, 1.3662...","[0.8442162293292362, 0.7685278777342012, 0.841...","[1.0887781220530783, 0.788091068301226, 1.1397...","[0.5297179487179488, 1.3102564102564105, 1.093...","[1.259697672558586, 1.4584499720067183, 1.3296...",24.886111,34.351389,41.238889,53.424658,71.445714
97,17020,'Folgaria',1114.698919,109.397429,15.104708,7.758886,222.742879,44.740514,109.598047,363.083485,...,"[0.4705553224227164, 0.7139460064344665, 0.892...","[0.7812032325651003, 0.9779507133592736, 1.186...","[1.0373983739837398, 1.2903178122690315, 1.350...","[1.2300503235082676, 1.304508575536613, 1.3908...","[1.5788071737424971, 1.2973308277535534, 1.149...",24.651724,34.562069,46.655172,67.151724,94.501724
98,17024,'Passo_Sommo',1323.954513,76.199199,14.777693,8.285551,156.549350,90.509941,107.036664,371.702008,...,"[0.9776579352850536, 0.6379044684129429, 1.802...","[1.1775244299674268, 0.5765472312703583, 1.548...","[1.2024840045163718, 0.5284155062100113, 1.077...","[1.515576129112974, 0.5720005004378832, 0.7476...","[1.419747220374418, 0.59

In [75]:
from sklearn.model_selection import train_test_split

#drop the test set from df_unique and create the train set and validation set

df_unique.drop(df_unique[df_unique['ID'].isin(test_ID)].index, inplace = True)

validation_split = 0.1
train_df, val_df = train_test_split(df_unique, test_size=validation_split, random_state=42)

/var/folders/73/q15nk7zj12b12wgzsnb2wkwm0000gn/T/ipykernel_1415/3043764305.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(df_unique[df_unique['ID'].isin(test_ID)].index, inplace = True)


In [76]:
train_df.reset_index(drop=True, inplace=True)
train_df

,ID,Nome,HMerit_mean(m),HMerit_stdev(m),Slope_mean(deg),Slope_stdev(deg),Aspect_mean(deg),Aspect_stdev(deg),MinDistAdriaticC_(km),OrogrAdri_mean (m),...,AMS_1,AMS_3,AMS_6,AMS_12,AMS_24,IdD_1,IdD_3,IdD_6,IdD_12,IdD_24
0,9380,'Melegnano',90.964295,3.898290,0.858622,0.606050,152.281257,104.700774,233.903377,46.875647,...,"[0.6349206349206349, 1.6349206349206349, 0.333...","[1.0093665269903869, 1.5942814887848165, 0.357...","[0.8698710433763188, 1.3007033997655333, 0.328...","[0.8308537422751201, 1.080949111161975, 0.3860...","[0.7887025375646861, 0.9217532265103808, 0.477...",31.500000,38.638095,48.742857,59.577273,72.904545
1,12163,'Fossano_2',360.764973,9.797826,1.848248,2.946933,189.374908,118.169549,359.137335,341.587794,...,"[0.6105197244834064, 0.5535378835316218, 1.335...","[1.0581789720845292, 0.7902426297938951, 1.261...","[1.4354166666666668, 1.2620833333333334, 1.278...","[1.1637931034482758, 1.254310344827586, 1.1120...","[1.3804123711340206, 0.9984536082474228, 1.005...",24.569231,29.484615,36.923077,46.400000,59.692308
2,9014,'Barni',767.637133,124.744350,22.700332,12.193675,189.562921,99.345678,246.645541,497.279626,...,"[1.2402402402402402, 0.6586586586586587, 0.907...","[0.9819233550253073, 0.89587852494577, 1.14135...","[0.9625839462743844, 0.7991685321394308, 1.434...","[0.9029680365296804, 0.946917808219178, 1.3384...","[0.7622596153846153, 0.9709134615384616, 1.156...",57.085714,79.028571,89.342857,100.114286,118.857143
3,21018,'Brendola',157.061393,90.859242,11.222639,8.708569,187.641838,59.111240,68.528559,36.508654,...,"[1.1390024529844645, 0.5437448896156991, 0.658...","[0.983451536643026, 0.6477541371158392, 0.7754...","[1.1950984441690762, 0.5705631281839461, 0.682...","[1.272998932764141, 0.63948772678762, 0.833724...","[1.4129561725161517, 0.5426925091671032, 0.738...",34.942857,42.300000,51.878571,66.928571,81.814286
4,8072,'Colle_Del_Melogno',1017.392522,148.313182,20.703809,7.836887,164.489004,67.864609,325.281240,363.897139,...,"[0.7110542040119504, 0.5825864276568501, 0.878...","[0.740983606557377, 0.4918032786885246, 0.9245...","[0.7797584818861414, 0.5290396779758482, 0.832...","[0.7836943762672073, 0.5497812399957316, 0.921...","[0.8176433878507423, 0.5367433207342777, 0.876...",33.471429,61.000000,86.950000,117.137500,151.655357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,16116,'Pisa_(Porta_a_Lucca)',4.118918,2.136247,0.398842,0.269551,180.744088,127.876986,168.190032,242.901101,...,"[0.9685641461342396, 1.1979609175870858, 0.943...","[0.8637640449438202, 1.601123595505618, 0.8890...","[0.8023186088346992, 1.4931041375174896, 0.859...","[0.736659845742169, 1.175822446088462, 0.79332...","[0.7724867724867726, 1.119047619047619, 0.7407...",39.233333,47.466667,55.588889,70.588889,75.600000
2022,12040,'Arborio',177.960379,2.677747,0.516301,0.331835,172.063822,64.690719,307.525001,77.824089,...,"[0.7877492877492878, 0.717948717948718, 0.5982...","[0.8476968796433879, 0.87369985141159, 0.43164...","[0.6761363636363636, 0.8352272727272727, 0.485...","[0.7040816326530611, 0.7891156462585033, 0.625...","[0.7812971342383107, 0.7892156862745098, 0.649...",20.057143,38.457143,50.285714,58.800000,75.771429
2023,12078,'Castagnole_Lanze',224.492772,27.519951,5.287030,3.568005,170.420486,127.937669,321.713922,240.215057,...,"[0.8407033170178351, 1.3431926559250469, 0.705...","[0.7227237019579943, 1.036358516015237, 0.8999...","[0.9889401141819744, 1.115304684327449, 0.9889...","[1.4631243109285512, 1.050218063672641, 0.8976...","[1.685302970458848, 0.93662748729962, 0.949157...",20.696957,29.333478,36.402609,44.562174,63.846087
2024,12253,'Corio',560.118121,50.829277,9.402977,6.183060,128.526813,69.854906,372.752970,90.497170,...,"[0.4810839395715123, 1.2828905055240327, 1.205...","[0.3984021304926764, 1.0575765645805593, 0.938...","[0.6083891946881755, 0.9048631169727688, 0.799...","[0.6219951014483989, 1.1668833721387315, 0.665...","[0.4880774771726369, 1.22

In [77]:
val_df.reset_index(drop=True, inplace=True)
val_df

,ID,Nome,HMerit_mean(m),HMerit_stdev(m),Slope_mean(deg),Slope_stdev(deg),Aspect_mean(deg),Aspect_stdev(deg),MinDistAdriaticC_(km),OrogrAdri_mean (m),...,AMS_1,AMS_3,AMS_6,AMS_12,AMS_24,IdD_1,IdD_3,IdD_6,IdD_12,IdD_24
0,16070,'Passo_di_Sella_auto',731.592133,111.887173,20.799196,9.850323,169.495478,112.725506,159.922705,470.226813,...,"[1.3362676056338028, 1.1038732394366195, 0.559...","[1.28601108033241, 1.298476454293629, 0.415512...","[1.3764906303236797, 1.0647359454855196, 0.558...","[1.455140186915888, 0.8761682242990655, 0.6686...","[1.31614539306847, 1.0574809805579035, 0.62637...",37.866667,96.266667,117.400000,142.666667,157.733333
1,9005,'Alpe_Entova',1976.916676,165.354018,21.590841,9.120648,204.564195,42.282189,221.394389,899.181916,...,"[0.7683823529411764, 0.859375, 1.4457720588235...","[0.8122886133032694, 1.109921082299887, 1.2277...","[0.8442714126807565, 1.1256952169076753, 1.162...","[0.8518962075848304, 1.2924817032601463, 1.100...","[0.6889012414304242, 1.2514359829534925, 1.222...",19.781818,32.254545,49.036364,68.318182,98.127273
2,9194,'Mezzoldo',1130.258751,211.621521,30.496304,10.763869,139.290865,75.060703,220.397143,723.668441,...,"[0.2256446991404011, 1.6869627507163325, 0.884...","[0.3340757238307349, 1.6035634743875278, 0.698...","[0.4114942528735631, 1.3241379310344827, 0.650...","[0.4360100376411543, 1.11198243412798, 0.75125...","[0.4813233724653148, 0.903948772678762, 0.7918...",27.920000,35.920000,43.500000,63.760000,93.700000
3,12120,'Salussola',230.548920,15.897973,2.264714,3.861750,141.209894,71.444404,328.157635,78.895274,...,"[1.0963455149501662, 1.036544850498339, 0.6378...","[0.970401691331924, 1.0845665961945032, 0.9133...","[0.9341021416803952, 0.8451400329489291, 0.869...","[0.9596928982725528, 0.8483685220729367, 0.871...","[0.9747675962815406, 0.9296148738379816, 0.937...",20.066667,31.533333,40.466667,52.100000,75.300000
4,9351,'Asso',563.576016,103.428747,18.522924,10.480815,174.302482,86.919004,245.574346,425.781297,...,"[1.4258281325012, 0.8079692750840134, 1.874699...","[1.253648366921473, 1.005211952744962, 1.45239...","[1.0287046347929427, 0.880884907212725, 1.1704...","[1.0079712189616252, 1.0196106094808126, 1.478...","[1.234431090994225, 1.2996722334946154, 1.6567...",37.872727,52.327273,66.296970,85.915152,116.490909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,12316,'Vinadio_Centrale',930.881556,138.480600,17.448311,14.513991,151.857907,131.786391,402.963771,445.790291,...,"[2.347550229115263, 1.825872400422982, 1.43461...","[1.9298706991014685, 1.4595660749506905, 1.662...","[1.5317020621729764, 1.3096337334564483, 1.822...","[1.593784078516903, 1.081352235550709, 1.59781...","[1.5952062926684478, 0.7413178984861978, 1.403...",15.335135,24.664865,35.124324,49.567568,72.843243
222,16217,'Pistoia',87.233911,8.262350,1.528556,1.257805,166.914298,62.236666,121.520200,356.959974,...,"[1.2000987735616098, 0.5704173182978022, 0.629...","[0.9645041559935176, 0.4940143238015578, 0.536...","[0.7630867535020889, 0.4055050380928975, 0.597...","[0.6572685200380741, 0.8389404929924181, 0.552...","[0.7080166969924008, 0.7032002568768062, 0.606...",26.997778,42.508889,54.253333,67.704444,83.048889
223,16155,'Serraglio',147.170333,35.416366,5.763455,4.952756,228.325846,88.132392,93.535229,462.322094,...,"[1.5274151436031331, 1.0966057441253263, 0.822...","[1.3968350578210589, 0.7668898356664637, 0.843...","[1.2246169826019215, 0.911451571020514, 0.8179...","[1.0241042345276874, 1.0905537459283388, 1.098...","[0.9116927132366786, 1.2540011297307474, 1.067...",25.533333,36.511111,42.788889,51.166667,59.011111
224,5149,'Lavino_Di_Sopra',76.547898,10.081892,1.688252,1.697579,162.535054,134.322513,83.852730,15.375321,...,[1.0],[1.0],[1.0],[1.0],[1.0],60.200000,83.200000,96.400000,113.200000,130.400000
